In [1]:
# Importing Libraries:
import pandas as pd
import numpy as np 

import findspark
from pyspark import SparkContext
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql import SparkSession
from pyspark.ml.feature import  StringIndexer
from pyspark.ml.classification import LogisticRegression
from sklearn.metrics import accuracy_score
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline


import findspark
findspark.init()

In [2]:
# Reading the data
data = pd.read_csv('cleaned.csv', encoding= 'unicode_escape') 
data=data.drop(['Unnamed: 0'], axis=1)
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,duration_of_stay
0,Renaissance Hotel,0,13,2018,July,27,1,0,1,1,...,A,0,No Deposit,304.0,0,Transient,75.00,0,0,1
1,Renaissance Hotel,0,14,2018,July,27,1,0,2,2,...,A,0,No Deposit,240.0,0,Transient,98.00,0,1,2
2,Renaissance Hotel,0,9,2018,July,27,1,0,2,2,...,C,0,No Deposit,303.0,0,Transient,103.00,0,1,2
3,Renaissance Hotel,1,85,2018,July,27,1,0,3,2,...,A,0,No Deposit,240.0,0,Transient,82.00,0,1,3
4,Renaissance Hotel,1,75,2018,July,27,1,0,3,2,...,D,0,No Deposit,15.0,0,Transient,105.50,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74844,JW Marriott Hotel,0,23,2017,August,35,30,2,5,2,...,A,0,No Deposit,394.0,0,Transient,96.14,0,0,7
74845,JW Marriott Hotel,0,102,2017,August,35,31,2,5,3,...,E,0,No Deposit,9.0,0,Transient,225.43,0,2,7
74846,JW Marriott Hotel,0,34,2017,August,35,31,2,5,2,...,D,0,No Deposit,9.0,0,Transient,157.71,0,4,7
74847,JW Marriott Hotel,0,109,2017,August,35,31,2,5,2,...,A,0,No Deposit,89.0,0,Transient,104.40,0,0,7


In [3]:
#Total number of duplicated rows--> no duplicated row 
data.duplicated().sum()


3

In [4]:
# Dropping duplicated rows
data=data.drop_duplicates()

In [5]:
# Checking if there are missing data
data.isnull().values.any()

False

In [6]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/06/08 19:44:53 WARN Utils: Your hostname, Lojins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
23/06/08 19:44:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/08 19:44:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/08 19:44:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# Loading the data into spark dataframe
df = spark.createDataFrame(data) 
# Checking that columns were inserted correctly 
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: long (nullable = true)
 |-- lead_time: long (nullable = true)
 |-- arrival_date_year: long (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: long (nullable = true)
 |-- arrival_date_day_of_month: long (nullable = true)
 |-- stays_in_weekend_nights: long (nullable = true)
 |-- stays_in_week_nights: long (nullable = true)
 |-- adults: long (nullable = true)
 |-- children: double (nullable = true)
 |-- babies: long (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: long (nullable = true)
 |-- previous_cancellations: long (nullable = true)
 |-- previous_bookings_not_canceled: long (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullable = true)
 |-- booking_changes: long (nullable = true)
 |-- deposit_type: string 

Model One: Multi-Layer Perceptron

In [8]:
label =StringIndexer(inputCol="is_canceled", outputCol="label")
model1 = label.fit(df).transform(df)

23/06/08 19:45:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
numericCols =['lead_time','arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','stays_in_weekend_nights','stays_in_week_nights','adults','children','babies','is_repeated_guest','previous_cancellations','previous_bookings_not_canceled','booking_changes','agent','days_in_waiting_list','adr','required_car_parking_spaces','total_of_special_requests']
assembler = VectorAssembler().setInputCols(numericCols).setOutputCol("features")
model1 = assembler.transform(model1)

In [10]:
train, test = model1.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 52296
Test Dataset Count: 22550


In [11]:
layers = [ 18, 14, 4, 2 ]
mlpc = MultilayerPerceptronClassifier(layers = layers).setLabelCol('label').setFeaturesCol('features')
model11 = mlpc.fit(train)

23/06/08 19:45:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/06/08 19:45:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/06/08 19:45:07 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
result = model11.transform(test)

In [13]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'accuracy')
mlpacc = evaluator.evaluate(result)
mlpacc
print("Multi-Layer Perceptron Accuracy =",mlpacc*100,"%")

Multi-Layer Perceptron Accuracy = 70.31929046563194 %


Model Two: Decision Tree Classifier

In [14]:
is_canceled_indexer = StringIndexer(inputCol="is_canceled", outputCol="is_canceled_index")
model2 = is_canceled_indexer.fit(df).transform(df)

In [15]:
numericCols =['lead_time','arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','stays_in_weekend_nights','stays_in_week_nights','adults','children','babies','is_repeated_guest','previous_cancellations','previous_bookings_not_canceled','booking_changes','agent','days_in_waiting_list','adr','required_car_parking_spaces','total_of_special_requests']
featurizationPipeline = Pipeline(stages = [VectorAssembler(inputCols=numericCols, outputCol="feature")])

In [16]:
pipelineModel = featurizationPipeline.fit(model2)
model2 = pipelineModel.transform(model2)

In [17]:
train, test = model2.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 52296
Test Dataset Count: 22550


In [18]:
dt = DecisionTreeClassifier(featuresCol = 'feature', labelCol = 'is_canceled_index', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

In [19]:
true_labels=predictions.select('is_canceled_index')
dt_predictions=predictions.select('prediction')

accuracy = accuracy_score(true_labels.toPandas(), dt_predictions.toPandas())
print("Logistic Regression Accuracy =",accuracy*100,"%")

Logistic Regression Accuracy = 70.38580931263859 %


Model Three: Random Forest Classifier

In [20]:
numericCols =['lead_time','arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','stays_in_weekend_nights','stays_in_week_nights','adults','children','babies','is_repeated_guest','previous_cancellations','previous_bookings_not_canceled','booking_changes','agent','days_in_waiting_list','adr','required_car_parking_spaces','total_of_special_requests']
featurizationPipeline = Pipeline(stages = [VectorAssembler(inputCols=numericCols, outputCol="feature_vector")])

In [21]:
featurizationPipelineModel = featurizationPipeline.fit(df)
model3 = featurizationPipelineModel.transform(df)

In [22]:
train, test = model3.randomSplit([0.8, 0.2], seed = 2018)

In [23]:
rf = RandomForestClassifier(featuresCol = 'feature_vector', labelCol = 'is_canceled')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [24]:
true_labels=predictions.select('is_canceled')
lr_predictions=predictions.select('prediction')

In [25]:
accuracy = accuracy_score(true_labels.toPandas(), lr_predictions.toPandas())
print("Random Forest Classifier Accuracy =",accuracy*100,"%")

Random Forest Classifier Accuracy = 73.64464844009845 %


Model Four: Logistic Regression

In [26]:
numericCols =['lead_time','arrival_date_year','arrival_date_week_number','arrival_date_day_of_month','stays_in_weekend_nights','stays_in_week_nights','adults','children','babies','is_repeated_guest','previous_cancellations','previous_bookings_not_canceled','booking_changes','agent','days_in_waiting_list','adr','required_car_parking_spaces','total_of_special_requests']
featurizationPipeline = Pipeline(stages = [VectorAssembler(inputCols=numericCols, outputCol="feature_vector")])

In [27]:
featurizationPipelineModel = featurizationPipeline.fit(df)
model4 = featurizationPipelineModel.transform(df)

In [28]:
train, test = model4.randomSplit([0.8, 0.2], seed = 2018)

In [29]:
lr = LogisticRegression(featuresCol = 'feature_vector', labelCol = 'is_canceled', maxIter=10)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)


In [30]:
true_labels=predictions.select('is_canceled')
lr_predictions=predictions.select('prediction')

In [31]:
accuracy = accuracy_score(true_labels.toPandas(), lr_predictions.toPandas())
print("Logistic Regression Accuracy =",accuracy*100,"%")

Logistic Regression Accuracy = 73.72447282644849 %
